In [15]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [16]:
df = pd.read_csv(Path('./data/20210926.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice]
0,21/09/2021 20:48:32,Wendy and Lucy (2010),Best in Show (2000),Beverly Hills Chihuahua (2008),OOPS ALL JOHN WATERS,[Default to Trish Pic],CENTRAL EUROPE
1,21/09/2021 20:51:12,Wendy and Lucy (2010),Beverly Hills Chihuahua (2008),NaN,OOPS ALL JOHN WATERS,CENTRAL EUROPE,[Default to Trish Pic]
2,22/09/2021 15:33:41,Best in Show (2000),NaN,NaN,CENTRAL EUROPE,OOPS ALL JOHN WATERS,[Default to Trish Pic]
3,23/09/2021 08:27:54,Beverly Hills Chihuahua (2008),Wendy and Lucy (2010),Best in Show (2000),CENTRAL EUROPE,OOPS ALL JOHN WATERS,[Default to Trish Pic]


In [17]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [18]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Wendy and Lucy (2010)', 'Best in Show (2000)',
       'Beverly Hills Chihuahua (2008)'], dtype=object)

In [19]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [20]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [21]:
submission_dict

{'Wendy and Lucy (2010)': <Candidate('Wendy and Lucy (2010)')>,
 'Best in Show (2000)': <Candidate('Best in Show (2000)')>,
 'Beverly Hills Chihuahua (2008)': <Candidate('Beverly Hills Chihuahua (2008)')>}

In [22]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [23]:
ballots

[<Ballot(Wendy and Lucy (2010), Best in Show (2000), Beverly Hills Chihuahua (2008))>,
 <Ballot(Wendy and Lucy (2010), Beverly Hills Chihuahua (2008))>,
 <Ballot(Best in Show (2000))>,
 <Ballot(Beverly Hills Chihuahua (2008), Wendy and Lucy (2010), Best in Show (2000))>]

In [24]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [25]:
print(election_result)

FINAL RESULT
Candidate                         Votes  Status
------------------------------  -------  --------
Wendy and Lucy (2010)                 2  Elected
Best in Show (2000)                   1  Rejected
Beverly Hills Chihuahua (2008)        1  Rejected



In [26]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

FINAL RESULT
Candidate                 Votes  Status
----------------------  -------  --------
OOPS ALL JOHN WATERS          2  Elected
CENTRAL EUROPE                2  Rejected
[Default to Trish Pic]        0  Rejected

